# First thing First "Lets import the required Libraries"

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder

Read The dataset to see the shape and size

In [1]:
DATA=pd.read_csv('../input/indian-candidates-for-general-election-2019/LS_2.0.csv')


print(DATA.size)
print(DATA.shape)

Let's see the columns and its attributes

In [1]:
DATA.info()

Rename,replace & change data type for column CRIMINAL CASES as there is a newline charecter in the 

In [1]:
DATA.rename(columns = {'TOTAL\nVOTES':'TOTALVOTES','CRIMINAL\nCASES':'CRIMINALCASES'}, inplace = True)

DATA['CRIMINALCASES'].fillna(DATA['CRIMINALCASES'].mode().values[0], inplace = True)


Winner Column is represented with  neumeric data , changing it  to Yes,No.

Let's see the number of Constituency across state 

In [1]:
CONS=DATA.groupby("STATE")["CONSTITUENCY"].nunique()
CONS.plot(figsize=(8,8));
CONS.plot.bar();

In [1]:
DATA.loc[DATA.WINNER==1,'WINNER']="Yes"
DATA.loc[DATA.WINNER==0,'WINNER']="No"

> No of Winners all over India

In [1]:
DATA['WINNER'].value_counts()

# Let's EXTRACT TAMILNADU DATA & See the number of Constituencies 

In [1]:
TAMIL_NADU=DATA[DATA['STATE']== 'Tamil Nadu']
TAMIL_NADU.head()
No_Of_Cons=TAMIL_NADU['CONSTITUENCY'].nunique()
No_Of_Cons

Lets have a look on  Tamil Nadu Election Results Lok-Sabha 2019 

In [1]:
TAMIL_NADU.head()

What's the total number of voters in TN

In [1]:
print("Number of Voters:",TAMIL_NADU['TOTAL ELECTORS'].sum())

FIVE POINT SUMMARY for nuemeric data****

In [1]:
TAMIL_NADU.describe().T

How Many Parties Contested in how many seats

In [1]:
TAMIL_NADU['PARTY'].value_counts()

Distribution of Contestent's Gender among parties/Among Winners 

In [1]:
TAMIL_NADU['GENDER'].value_counts()
Gender_tn=pd.crosstab(TAMIL_NADU['PARTY'],TAMIL_NADU['GENDER'])
Gender_tn.plot.bar();
plt.title("No of Male and Female Candidates across party in Tamil Nadu 2019")
Gender_win=pd.crosstab(TAMIL_NADU['GENDER'],TAMIL_NADU['WINNER'])
Gender_win.plot.bar();
plt.title("Contestents splitup based on result 2019 Tamil Nadu")
Gender_win_ind=pd.crosstab(DATA['GENDER'],DATA['WINNER'])
Gender_win_ind.plot.bar();
plt.title("Contestents splitup based on result 2019 All Over India")

**Check Data Skewness

In [1]:
TAMIL_NADU.skew()

Does Gender has an influence in the result of a Candidate in 2019 in Tamil Nadu??? 

In [1]:
sns.scatterplot(TAMIL_NADU['PARTY'],TAMIL_NADU['GENDER'],hue=TAMIL_NADU['WINNER']);

From The Above plot we are not sure if GENDER has a role in result, Let's find out through a chi square test ...

In [1]:
#Let's define the Null Hypothesis & Alternate Hypothesis
H0="Gender does not have any impact on winning in 2019"
Ha="Gender does have an  impact on  winning in 2019"
chi,p_value,expected,dof=stats.chi2_contingency(Gender_win_ind)
print (p_value)
if p_value < 0.05:
    print(f'{Ha} as the p_value ({p_value}) < 0.05')
else:
    print(f'{H0} as the p_value ({p_value}) > 0.05')

So, Gender did not have an impact in TN election reslts. How about "Education" ??? we shall TEST THE SIGNIFICANCE OF Education with the result of the candidate

In [1]:
sns.scatterplot(x=TAMIL_NADU['PARTY'],y=TAMIL_NADU['TOTALVOTES'],hue=TAMIL_NADU['EDUCATION']);

    I start liking the Chi square test :)

In [1]:
TAB=pd.crosstab(TAMIL_NADU['WINNER'],TAMIL_NADU['EDUCATION'])
TAB
H0="Education of Candidates does not influence winning"
Ha="Education of candidates does influence winning"
chi,p_value,dof,expected=stats.chi2_contingency(TAB)
print(p_value)
if p_value < 0.05:
    print(Ha)
else:
    print(H0)

Oop's seems like Education is not being a factor for a contestent

Vote Share of Top 5 Parties in Indian Parliment Election 2019- All Over India

In [1]:
Vote_share=DATA.groupby("PARTY")["TOTALVOTES"].sum().nlargest(5).index.tolist()
Vote_share
def fuc(row):
    if row["PARTY"] not in Vote_share:
        return("Others")
    else:
        return row['PARTY']
DATA['Party_new']=DATA.apply(fuc,axis=1)
DATA.head()
Top5=DATA.groupby("Party_new")["TOTALVOTES"].sum()
Top5_index=Top5.index
Top5_label=Top5.values
Top5.plot.pie(labels=Top5_index,
        shadow=True, startangle=30);

Vote Share of Top 5 Parties in Indian Parliment Election 2019- Tamil Nadu

In [1]:
TOP5_TN=TAMIL_NADU.groupby("PARTY")["TOTALVOTES"].sum().nlargest(5).index.tolist()
TOP5_TN
def TN(row):
    if row["PARTY"] not in TOP5_TN:
        return("Others")
    else:
        return row["PARTY"]
TAMIL_NADU['TN']=TAMIL_NADU.apply(TN,axis=1)
TN_pie=TAMIL_NADU.groupby('TN')["TOTALVOTES"].sum()
TN_pie.plot.pie()

Hi, Thanks for visiting. I have Just done some ***EDA and hypothesis testing*** on this dataset no predictions have been done. Just exploring the data on various parameters. If you like it do upvote. 
"Happy Learning!"